## Algoritmo de automatización 



In [16]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
# !!! PLEASE UPDATE THESE PATHS TO YOUR ACTUAL FILE LOCATIONS !!!
RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values.json" # New output name v4

# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05 # Lower practical limit for TDS
MAX_TDS = 1
MIN_PICKUP = 0.05
# Stricter pickup limit relative to Ishc, inspired by snippet
MAX_PICKUP_FACTOR = 0.7
# MAX_TIME still needed for constraint violations, but not for calculation errors
MAX_TIME = 20.0
MAX_ITERATIONS = 250 # More iterations might be needed
# Convergence Targets
TARGET_TMT = -0.005 # Target for *total* miscoordination time (can stay negative)
# ***MODIFIED: New threshold for individual Margin Time (MT)***
MIN_ALLOWED_INDIVIDUAL_MT = -0.009 # Convergence threshold for the most negative individual MT
CONVERGENCE_THRESHOLD_TMT = 0.005 # Smaller margin for TMT (still used for stagnation check)
# Objective Function Weights
W_TIME = 0.1 # Further decrease weight of total time
W_MT = 15.0 # Increase weight for miscoordination penalty
W_PICKUP_DIFF = 0.0
# Adjustment Step Sizes & Factors (Inspired by Snippet)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75 # Threshold to trigger aggressive adjustments
# Aggressive Steps
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15 # Increase backup TDS more
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90  # Decrease main TDS more
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05 # Increase backup pickup if TDS maxed out
# Normal Steps (additive for TDS, small mult for pickup)
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01
NORMAL_PICKUP_MAIN_FACTOR = 0.99


# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

# MODIFIED: Returns Optional[float] - None indicates calculation error
def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    """Calculates relay operation time. Returns None if calculation is impossible."""
    # Basic Input Validation / Conditions for non-calculation
    if I_pi <= 0 or I_shc <= 0 or TDS < MIN_TDS or TDS > MAX_TDS:
        # logger.debug(f"Calc Error: Invalid input I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}")
        return None # Return None for impossible basic inputs

    # Pickup Constraint Check (Violation leads to MAX_TIME, not None)
    # Allow pickup up to Ishc*factor. If it exceeds, it's a constraint violation.
    if I_pi > I_shc * MAX_PICKUP_FACTOR:
        # logger.debug(f"Calc Constraint Violation: I_pi={I_pi} > I_shc*factor={I_shc * MAX_PICKUP_FACTOR}")
        return MAX_TIME # Return MAX_TIME for pickup constraint violation

    M = I_shc / I_pi
    # Check for M close to or below 1 (prevents division by zero/negative log)
    # If pickup is *valid* but M <= 1, it means the relay shouldn't operate -> MAX_TIME
    if M <= 1.0:
        # logger.debug(f"Non-operation: M={M} <= 1.0 (I_shc={I_shc}, I_pi={I_pi})")
        return MAX_TIME # Return MAX_TIME if M indicates non-operation

    try:
        denominator = M**N - 1
        # Check for near-zero denominator (should be less likely now M > 1 is enforced)
        if abs(denominator) < 1e-9:
            # logger.debug(f"Calc Error: Denominator near zero ({denominator})")
            return None # Return None for division by zero (potential numerical instability)

        time = TDS * (K / denominator)

        # Check for non-finite or non-positive results
        if not np.isfinite(time) or time <= 0:
             # logger.debug(f"Calc Error: Result non-finite or non-positive ({time})")
             return None # Return None for invalid results

        return min(time, MAX_TIME) # Return calculated time, capped by MAX_TIME

    except (OverflowError, ValueError) as e:
        # logger.debug(f"Calc Error: Math exception M={M}, N={N} -> {e}")
        return None # Return None on math errors
    except Exception as e:
        logger.error(f"Excepción inesperada en calc_op_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None # Return None for any other unexpected error

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay'); backup_relay_info = pair_entry.get('backup_relay')
        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            # logger.warning(f"Omitiendo entrada de par {i+1}: Falta scenario_id o info de relé inválida.")
            skipped_pairs_count+=1; continue
        if scenario_id not in scenario_map: scenario_map[scenario_id] = {"pairs_info": [], "initial_settings": {}, "relays": set()}
        main_relay = main_relay_info.get('relay'); backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc'); I_shc_backup = backup_relay_info.get('Ishc')
        # Ensure Ishc are positive numbers for calculations
        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            # logger.warning(f"Omitiendo par {main_relay}/{backup_relay} en {scenario_id}: Falta nombre de relé o Ishc inválido/no positivo (Main: {I_shc_main}, Backup: {I_shc_backup}).")
            skipped_pairs_count+=1; continue

        processed_pairs_count += 1
        scenario_map[scenario_id]['relays'].add(main_relay); scenario_map[scenario_id]['relays'].add(backup_relay)
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                tds = r_info.get('TDS'); pickup = r_info.get('pick_up') # Key is 'pick_up' in input
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup)
                         }
                # else:
                    # logger.warning(f"({scenario_id}) Config inicial TDS/pickup no encontrada o inválida para '{r_name}' en la entrada de par. Se usarán valores por defecto si es necesario más adelante.")
        scenario_map[scenario_id]['pairs_info'].append({"main_relay": main_relay, "backup_relay": backup_relay, "I_shc_main": float(I_shc_main), "I_shc_backup": float(I_shc_backup)})

    logger.info(f"Datos agrupados por escenario. Pares procesados: {processed_pairs_count}, Pares omitidos: {skipped_pairs_count}")
    return scenario_map


# --- Scenario-Specific Optimization Function (MODIFIED for None handling and new convergence) ---
def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]], # Contains TDS_initial, pickup_initial
    relays_in_scenario: Set[str]
) -> Dict[str, Dict[str, float]]:
    """Performs iterative optimization, handles calculation errors (None), excludes errors from TMT, and uses the new individual MT convergence threshold."""

    logger.info(f"--- Iniciando optimización para {scenario_id} ---")
    if not pairs_info: logger.warning(f"({scenario_id}) No hay pares válidos."); return {}

    # 1. Initialize settings - Use initial Pickup, MIN TDS
    relay_settings = {}
    default_pickup = MIN_PICKUP * 1.5 # Start slightly above min

    for relay in relays_in_scenario:
         if relay in initial_settings:
             relay_settings[relay] = {
                 "TDS": MIN_TDS,
                 "pickup": max(MIN_PICKUP, initial_settings[relay]['pickup_initial'])
             }
         else:
             logger.warning(f"({scenario_id}) Config inicial (pickup) no encontrada para '{relay}'. Usando defecto.")
             relay_settings[relay] = {"TDS": MIN_TDS, "pickup": default_pickup}

    # --- Optimization Loop ---
    last_tmt = float('inf')
    no_improvement_streak = 0

    for iteration in range(MAX_ITERATIONS):
        total_main_time = 0.0; tmt = 0.0; miscoordination_penalty = 0.0
        current_pair_results = []; max_neg_mt = 0 # Initialize to 0 (least negative possible)
        error_pairs_count = 0 # Count pairs with calculation errors

        # Calculate performance
        for pair in pairs_info:
            main_relay = pair["main_relay"]; backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]; I_shc_backup = pair["I_shc_backup"]
            if main_relay not in relay_settings or backup_relay not in relay_settings: continue # Should not happen with grouping logic

            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            # Apply pickup bounds BEFORE calculating time
            # MAX_PICKUP_FACTOR constraint is checked *inside* calculate_operation_time
            pickup_main_bounded = max(MIN_PICKUP, pickup_main)
            pickup_backup_bounded = max(MIN_PICKUP, pickup_backup)

            # Calculate times - MODIFIED: handle None return
            main_time = calculate_operation_time(I_shc_main, pickup_main_bounded, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup_bounded, tds_backup)

            delta_t: Optional[float] = None
            mt: Optional[float] = None

            # Check for calculation errors (None) first
            if main_time is None or backup_time is None:
                delta_t = None
                mt = None
                error_pairs_count += 1
                # logger.debug(f"({scenario_id}) Iter {iteration+1}: Error calc pair {main_relay}({main_time})/{backup_relay}({backup_time})")
            # Check for MAX_TIME (constraint violation or non-operation) next
            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                 # If one is MAX_TIME and the other isn't, coordination is definitely violated.
                 # If both are MAX_TIME, they don't coordinate, but it's not a typical miscoordination.
                 # Assign a large penalty value to mt to indicate a severe issue, but not infinity.
                 # Avoid using MAX_TIME directly for mt calculation if possible to distinguish from calc error
                 delta_t = backup_time - main_time if main_time < MAX_TIME and backup_time < MAX_TIME else MAX_TIME
                 # If delta_t is MAX_TIME (or either time was MAX_TIME), assign a large penalty to mt
                 mt = MAX_TIME * 2 # Assign a large penalty if times are MAX_TIME
                 # logger.debug(f"({scenario_id}) Iter {iteration+1}: MAX_TIME detected for {main_relay}({main_time})/{backup_relay}({backup_time}) -> mt={mt}")
            # Otherwise, calculate normally
            else:
                 delta_t = backup_time - main_time
                 mt = delta_t - CTI

            current_pair_results.append({
                "main_time": main_time, "backup_time": backup_time, # Can be None, float, or MAX_TIME
                "delta_t": delta_t, "mt": mt, # Can be None, float, or large penalty value
                "main_relay": main_relay, "backup_relay": backup_relay,
                "I_shc_main": I_shc_main, "I_shc_backup": I_shc_backup
            })

            # --- Accumulate performance metrics - MODIFIED: Skip None values for TMT/Penalty ---
            # Add main time only if valid and finite
            if main_time is not None and main_time < MAX_TIME:
                total_main_time += main_time

            # Add to TMT, penalty, and track max_neg_mt ONLY if mt is a valid negative number
            # Exclude the large penalty values assigned for MAX_TIME cases from normal TMT/max_neg_mt
            if mt is not None and mt < 0 and mt < MAX_TIME: # Check mt is negative AND not the large penalty
                tmt += mt
                miscoordination_penalty += mt**2
                max_neg_mt = min(max_neg_mt, mt) # Update the most negative MT found so far
            # --- End Accumulation Modification ---

        # Objective function calculation remains the same structure
        of = W_MT * miscoordination_penalty + W_TIME * total_main_time

        if (iteration + 1) % 25 == 0 or iteration == 0 or tmt > TARGET_TMT*1.1 :
             logger.info(f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

        # --- Check Convergence (MODIFIED: using MIN_ALLOWED_INDIVIDUAL_MT) ---
        # Converge if the total miscoordination is below target AND the worst individual miscoordination is above the minimum allowed limit
        if tmt >= TARGET_TMT and max_neg_mt >= MIN_ALLOWED_INDIVIDUAL_MT:
             logger.info(f"({scenario_id}) Convergencia alcanzada (TMT={tmt:.4f} >= {TARGET_TMT}, MaxNegMT={max_neg_mt:.4f} >= {MIN_ALLOWED_INDIVIDUAL_MT}).")
             break
        # --- End Convergence Check Modification ---

        # --- AJUSTE V3 (MODIFIED: Skip pairs with mt=None or mt indicating MAX_TIME penalty) ---
        adjustments_made = False
        next_relay_settings = copy.deepcopy(relay_settings)

        for pair_res in current_pair_results:
            # --- ADDED: Skip adjustment if mt could not be calculated or indicates MAX_TIME penalty ---
            if pair_res["mt"] is None or pair_res["mt"] >= MAX_TIME :
                continue
            # --- End Skip ---

            # Adjust only miscoordinated pairs (mt < 0 AND mt is not the large penalty value)
            if pair_res["mt"] < 0:
                main_relay = pair_res["main_relay"]; backup_relay = pair_res["backup_relay"]
                I_shc_main = pair_res["I_shc_main"]; I_shc_backup = pair_res["I_shc_backup"]
                mt_val = pair_res["mt"] # We know mt_val is a valid negative float here

                tds_main_curr = next_relay_settings[main_relay]["TDS"]
                tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
                pickup_main_curr = next_relay_settings[main_relay]["pickup"]
                pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

                new_tds_backup = tds_backup_curr
                new_pickup_backup = pickup_backup_curr
                new_tds_main = tds_main_curr
                new_pickup_main = pickup_main_curr

                # Apply adjustments based on severity
                if mt_val < AGGRESSIVE_MT_THRESHOLD:
                    # Aggressive adjustment
                    # logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Agresivo para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, tds_backup_curr * AGGRESSIVE_TDS_BACKUP_FACTOR)
                    # Only adjust pickup if TDS hits max and miscoordination is still severe
                    if abs(new_tds_backup - MAX_TDS) < 1e-6:
                         # Ensure pickup adjustment respects the MAX_PICKUP_FACTOR limit relative to *its* Ishc
                         max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                         new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * AGGRESSIVE_PICKUP_BACKUP_FACTOR))

                    new_tds_main = max(MIN_TDS, tds_main_curr * AGGRESSIVE_TDS_MAIN_FACTOR)
                    # Aggressively decrease main pickup only if main TDS hits min? (Optional refinement)
                    # if abs(new_tds_main - MIN_TDS) < 1e-6:
                    #    max_allowed_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
                    #    new_pickup_main = min(max_allowed_pickup_main, max(MIN_PICKUP, pickup_main_curr * 0.98)) # Example factor

                else:
                    # Normal adjustment
                    # logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Normal para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, max(MIN_TDS, tds_backup_curr + NORMAL_TDS_BACKUP_ADD))
                    new_tds_main = min(MAX_TDS, max(MIN_TDS, tds_main_curr - NORMAL_TDS_MAIN_SUB))
                    # Subtle pickup adjustments (less critical now with MAX_PICKUP_FACTOR check in calc_time)
                    # max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                    # new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * NORMAL_PICKUP_BACKUP_FACTOR))
                    # max_allowed_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
                    # new_pickup_main = min(max_allowed_pickup_main, max(MIN_PICKUP, pickup_main_curr * NORMAL_PICKUP_MAIN_FACTOR))


                # Update the temporary settings dict, ensuring bounds are respected immediately
                next_relay_settings[backup_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_backup))
                next_relay_settings[main_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_main))
                # Apply pickup bounds during adjustment application
                max_pickup_b = I_shc_backup * MAX_PICKUP_FACTOR
                max_pickup_m = I_shc_main * MAX_PICKUP_FACTOR
                next_relay_settings[backup_relay]["pickup"] = min(max_pickup_b, max(MIN_PICKUP, new_pickup_backup))
                next_relay_settings[main_relay]["pickup"] = min(max_pickup_m, max(MIN_PICKUP, new_pickup_main))


                adjustments_made = True
        # --- End Adjustment Modification ---

        if adjustments_made:
            relay_settings = next_relay_settings # Apply the adjusted settings for the next iteration
        elif iteration > 15: # Stop if no adjustments needed after initial phase
             logger.info(f"({scenario_id}) No se realizaron ajustes en iteración {iteration + 1}. Deteniendo.")
             break

        # Check for stagnation (based on TMT from valid pairs)
        # Use the original CONVERGENCE_THRESHOLD_TMT for stagnation check magnitude
        if abs(tmt - last_tmt) < CONVERGENCE_THRESHOLD_TMT: # Check if TMT improvement is negligible
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0
        last_tmt = tmt

        if no_improvement_streak >= 25:
            logger.warning(f"({scenario_id}) TMT no mejora significativamente ({no_improvement_streak} iteraciones). Deteniendo.")
            break

    else: # Executes if loop finishes without break (max iterations reached)
        logger.warning(f"({scenario_id}) La optimización no convergió (o no mejoró) después de {MAX_ITERATIONS} iteraciones.")
        # Log final state if max iterations reached
        logger.warning(f"({scenario_id}) Estado final: TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")


    # Format final results
    formatted_settings = {}
    relays_max_ishc = {} # Cache max Ishc per relay for final bounding
    for relay in relays_in_scenario:
        max_ishc = 0
        for p in pairs_info:
             if p['main_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_main', 0))
             if p['backup_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_backup', 0))
        # Ensure there's a sensible Ishc even if a relay only appears with Ishc=0 somehow (use default pickup relation)
        relays_max_ishc[relay] = max_ishc if max_ishc > 0 else (default_pickup / MIN_PICKUP)

    for relay, settings in relay_settings.items():
        final_pickup = settings['pickup']
        max_ishc_relay = relays_max_ishc.get(relay, default_pickup / MIN_PICKUP) # Fallback Ishc

        # Apply final bounds using the max Ishc seen by this relay in this scenario
        # This ensures the final pickup setting is valid w.r.t the highest fault current it needs to handle
        final_pickup_bounded = min(max_ishc_relay * MAX_PICKUP_FACTOR, max(MIN_PICKUP, final_pickup))
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, settings['TDS']))

        formatted_settings[relay] = {
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }
    logger.info(f"--- Optimización finalizada para {scenario_id} ---")
    return formatted_settings


# --- Script Execution ---
if __name__ == "__main__":
    logger.info("--- Iniciando Script de Optimización de Ajustes de Relés (v4 - MT Threshold -0.009) ---")
    # !!! ENSURE PATHS ARE CORRECT !!!
    if not os.path.exists(RELAY_PAIRS_PATH):
         logger.error(f"¡Error Crítico! No se encontró el archivo de entrada: {RELAY_PAIRS_PATH}")
         raise SystemExit("Archivo de pares de relés no encontrado.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None: raise SystemExit("Error crítico al cargar archivo de pares.")

    scenario_data_map = group_data_by_scenario(relay_pairs_data)
    if not scenario_data_map: raise SystemExit("Error crítico al procesar datos de entrada. No se encontraron escenarios o pares válidos.")

    all_optimized_settings = {}
    successful_scenarios = 0
    failed_scenarios = 0
    for scenario_id, scenario_data in scenario_data_map.items():
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
             logger.warning(f"Omitiendo escenario '{scenario_id}': No contiene relés o pares de información válidos después del procesamiento inicial.")
             failed_scenarios += 1
             continue
        try:
            optimized_settings_for_scenario = run_scenario_optimization(
                scenario_id,
                scenario_data['pairs_info'],
                scenario_data['initial_settings'],
                scenario_data['relays']
            )
            if optimized_settings_for_scenario:
                 all_optimized_settings[scenario_id] = optimized_settings_for_scenario
                 successful_scenarios += 1
            else:
                 logger.warning(f"La optimización no produjo resultados para el escenario: {scenario_id}")
                 failed_scenarios += 1
        except Exception as e:
            logger.error(f"Error inesperado durante optimización del escenario {scenario_id}: {e}", exc_info=True)
            failed_scenarios += 1


    # Format output as list
    output_list = []
    if all_optimized_settings:
        logger.info(f"Optimización completada. Escenarios exitosos: {successful_scenarios}, Escenarios fallidos/omitidos: {failed_scenarios}.")
        logger.info("Formateando resultados optimizados en la estructura de lista deseada...")
        for scenario_id, optimized_settings in all_optimized_settings.items():
            # Use a consistent way to generate OID-like strings if needed for DB
            # generated_oid = ''.join(random.choices(string.hexdigits.lower(), k=24))
            # Or just use scenario_id and timestamp for uniqueness if OID isn't strictly required
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z') # Ensure Z format
            list_entry = {
                # "_id": { "$oid": generated_oid }, # Uncomment if MongoDB OID structure is needed
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": optimized_settings # Contains optimized {'RelayName': {'TDS': x, 'pickup': y}}
            }
            output_list.append(list_entry)
        logger.info(f"Formato de lista creado con {len(output_list)} escenarios optimizados.")

        # Save the list
        try:
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir: os.makedirs(output_dir, exist_ok=True)
            # Use allow_nan=False to ensure compatibility with standard JSON (NaN/Infinity are not valid JSON)
            # Python's None will be converted to JSON null
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w') as file:
                json.dump(output_list, file, indent=2, allow_nan=False)
            logger.info(f"Archivo con ajustes optimizados (v4, MT>=-0.009, null handling) guardado en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        except Exception as e:
             logger.error(f"Error al guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
    else:
        logger.error(f"La optimización falló o no produjo resultados para ningún escenario ({successful_scenarios} exitosos, {failed_scenarios} fallidos/omitidos). No se guardó ningún archivo.")

    logger.info("--- Script de Optimización Finalizado ---")

2025-04-20 14:09:04,550 - INFO - --- Iniciando Script de Optimización de Ajustes de Relés (v4 - MT Threshold -0.009) ---
2025-04-20 14:09:05,190 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-20 14:09:05,220 - INFO - Datos agrupados por escenario. Pares procesados: 6720, Pares omitidos: 80
2025-04-20 14:09:05,227 - INFO - --- Iniciando optimización para scenario_1 ---
2025-04-20 14:09:05,238 - INFO - (scenario_1) Iter 1/250: OF=136.4166, TMT=-23.8825, MaxNegMT=-0.6159, Errors=0
2025-04-20 14:09:05,266 - INFO - (scenario_1) Iter 25/250: OF=25.0857, TMT=-6.2208, MaxNegMT=-0.4648, Errors=0
2025-04-20 14:09:05,281 - INFO - (scenario_1) Iter 50/250: OF=13.6421, TMT=-3.0775, MaxNegMT=-0.2350, Errors=0
2025-04-20 14:09:05,302 - INFO - (scenario_1) Iter 75/250: OF=14.1032, TMT=-2.0754, MaxNegMT=-0.2381, Errors=0
2025-04-20 14:09:05,322 - INFO - (scenario_1) Iter 100/250: OF=13.9546, TMT=-1.0004, MaxN